In [1]:
import lightgbm as lgb
import sys
from os import listdir
import os.path
from os.path import isdir, isfile, join
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [2]:
def readCSVfromDirectory(dir):
    print(dir)
    filepaths = []
    for f in listdir(dir):
        path = join(dir, f)
        if isfile(path) and f.endswith(".csv"):
            filepaths.append(path)

    if len(filepaths) == 0:
        exit("Directory does not contain any csv files: %s" % dir)

    filepaths = np.sort(filepaths)
    print("Found %s files." % len(filepaths))

    return filepaths


In [3]:
#############################################
#MARK: - Read Files

SOURCE_DIR = "./data/"


SOURCE_DIR = os.path.abspath(SOURCE_DIR)
if not isdir(SOURCE_DIR):
    exit("That's not a directory!")
else:
    print(SOURCE_DIR)

elems = listdir(SOURCE_DIR)
print(elems)
directories = {}
for e in elems:
    path = join(SOURCE_DIR, e)
    
    if isdir(path):
        filepaths = readCSVfromDirectory(path)[:25]

        directories[e] = []
        for filepath in filepaths:
            # the files contain values in [V] and [A]
            # use a current Factor of 10e-3 if given in [mA]
            dataframe = pd.read_csv(filepath, header=1)
            # Felix ist schuld
            dataframe.columns = dataframe.columns.str.strip().str.replace('(', '').str.replace(')', '') 
            #create id field
            dataframe = dataframe.assign(id= lambda x: dataframe.index )
            #remove ms field
            dataframe = dataframe.drop(columns=['ms'])
            if 'mA' in dataframe:
                dataframe = dataframe.assign(A= lambda x: x.mA / 1000)
                dataframe = dataframe.drop(columns=['mA'])
                #print("Converted mA to A")
            directories[e].append(dataframe)
            

/Users/paul/DataScienceProjekt/data
['ohp_voll-single.psdata', 'laptop-single', 'ohp_voll-laptop', 'lampe-ohp_voll.psdata', 'ohp_halb-single', 'ohp_halb-single.psdata', 'ohp_voll-laptop-bosch-lampe.psdata', 'lampe-ohp_voll', 'ohp_voll-single', 'ohp_halb-laptop-lampe.psdata', 'ohp_halb-laptop.psdata', 'bosch-single', 'lampe-single', 'ohp_voll-laptop-bosch-lampe', 'ohp_halb-laptop', 'ohp_halb-laptop-lampe']
/Users/paul/DataScienceProjekt/data/laptop-single
Found 256 files.
/Users/paul/DataScienceProjekt/data/ohp_voll-laptop
Found 256 files.
/Users/paul/DataScienceProjekt/data/ohp_halb-single
Found 256 files.
/Users/paul/DataScienceProjekt/data/lampe-ohp_voll
Found 256 files.
/Users/paul/DataScienceProjekt/data/ohp_voll-single
Found 256 files.
/Users/paul/DataScienceProjekt/data/bosch-single
Found 256 files.
/Users/paul/DataScienceProjekt/data/lampe-single
Found 256 files.
/Users/paul/DataScienceProjekt/data/ohp_voll-laptop-bosch-lampe
Found 256 files.
/Users/paul/DataScienceProjekt/data/

In [4]:
for (id, dfs) in directories.items():
    print(id, len(dfs))
    print(dfs[0].head(4))
    print(dfs[0].describe())
    print('\n\n')

laptop-single 25
          V         A  id
0 -8.224738  0.018067   0
1 -8.224738  0.036012   1
2 -3.753777  0.018067   2
3  0.732444  0.018067   3
                 V            A           id
count  2445.000000  2445.000000  2445.000000
mean     38.888648     0.021008  1222.000000
std     222.962687     0.338291   705.955027
min    -321.848200    -1.293191     0.000000
25%    -178.472200    -0.017823   611.000000
50%      81.362350     0.000122  1222.000000
75%     251.609800     0.018067  1833.000000
max     314.340600     1.419172  2444.000000



ohp_voll-laptop 25
           V         A  id
0 -12.710960 -1.078524   0
1  -8.224738 -1.078524   1
2  -3.753777 -1.078524   2
3  -8.224738 -1.078524   3
                 V            A           id
count  2445.000000  2445.000000  2445.000000
mean     38.879487     0.443960  1222.000000
std     221.681810     2.560982   705.955027
min    -317.362000    -5.035249     0.000000
25%    -174.001300    -1.887875   611.000000
50%      81.362350   

#### TODO

- Zeit diskretisieren 0ms = 0 und 20ms = 2445 (neue spalte -> id des datensatzes) done
- mA zu A umrechnen (falls vorhanden) done
- geht multilabel mit gradient boosting? jaein, vielleicht
- in X und Y aufteilen 
- attacke!


In [5]:
features = []

for label, data in directories.items():
    # pro gerät alle daten in einen dataframe
    res = pd.concat(data, sort=False)
    #zu der tabelle eine spalte mit label hinzufügen
    res = res.assign(labels= lambda x: label)
    
    features.append(res)

# dataframes zu einem kombinieren
features = pd.concat(features, sort=False)

X = features.drop(columns=['labels'])
#min_max_scaler = preprocessing.MinMaxScaler()
#X = min_max_scaler.fit_transform(X)
Y = features['labels']

print(X)

Y = pd.get_dummies(Y)
Y = Y.assign(labels= lambda x: 0 + x['lampe-ohp_voll'] + x['lampe-single'] * 2 + x['laptop-single'] * 3 + x['ohp_halb-laptop']* 4 + x['ohp_halb-laptop-lampe'] * 5 + x['ohp_halb-single'] * 6 + x['ohp_voll-laptop'] * 7 + x['ohp_voll-laptop-bosch-lampe'] * 8 + x['ohp_voll-single'] *9)
Y = Y['labels']


              V         A    id
0     -8.224738  0.018067     0
1     -8.224738  0.036012     1
2     -3.753777  0.018067     2
3      0.732444  0.018067     3
4      0.732444  0.036012     4
5      0.732444  0.018067     5
6      9.689627  0.018067     6
7      9.689627  0.036012     7
8      9.689627  0.018067     8
9      9.689627  0.018067     9
10    14.175850  0.018067    10
11    18.646810  0.018067    11
12    18.646810  0.018067    12
13    23.117770  0.018067    13
14    23.117770  0.018067    14
15    27.603990  0.018067    15
16    27.603990  0.018067    16
17    27.603990  0.018067    17
18    32.074950  0.018067    18
19    36.561170  0.000122    19
20    36.561170  0.018067    20
21    36.561170  0.000122    21
22    41.047390  0.018067    22
23    45.518360  0.000122    23
24    45.518360  0.000122    24
25    45.518360  0.018067    25
26    50.004580  0.018067    26
27    50.004580  0.018067    27
28    54.475540  0.000122    28
29    54.475540  0.000122    29
...     

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, shuffle=True)

In [7]:
train_data = lgb.Dataset(X_train, label=Y_train)

test_data = lgb.Dataset(X_test, label=Y_test)

In [8]:
param = {'num_leaves':200, 'num_trees':100, 'objective':'multiclass', 'num_class': 10}
num_round = 10
param['metric'] = ['multi_logloss', 'multi_error']

#bst = lgb.cv(param, train_data, num_round, nfold=5)
bst = lgb.train(param, train_data, num_round, valid_sets=[test_data])

/Users/paul/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's multi_error: 0.254184	valid_0's multi_logloss: 1.99005
[2]	valid_0's multi_error: 0.228695	valid_0's multi_logloss: 1.78317
[3]	valid_0's multi_error: 0.217211	valid_0's multi_logloss: 1.62511
[4]	valid_0's multi_error: 0.205366	valid_0's multi_logloss: 1.49524
[5]	valid_0's multi_error: 0.201276	valid_0's multi_logloss: 1.3864
[6]	valid_0's multi_error: 0.189971	valid_0's multi_logloss: 1.29129
[7]	valid_0's multi_error: 0.186896	valid_0's multi_logloss: 1.20961
[8]	valid_0's multi_error: 0.184131	valid_0's multi_logloss: 1.13684
[9]	valid_0's multi_error: 0.179959	valid_0's multi_logloss: 1.07276
[10]	valid_0's multi_error: 0.177832	valid_0's multi_logloss: 1.01595
[11]	valid_0's multi_error: 0.175378	valid_0's multi_logloss: 0.963605
[12]	valid_0's multi_error: 0.173628	valid_0's multi_logloss: 0.916157
[13]	valid_0's multi_error: 0.172695	valid_0's multi_logloss: 0.874457
[14]	valid_0's multi_error: 0.171207	valid_0's multi_logloss: 0.836021
[15]	valid_0's multi_error